## Generate a SWOT swath.

This example lets us understand how to initialize the simulation parameters (error, SSH interpolation, orbit), generate an orbit, generate a swath, interpolate the SSH, and simulate the measurement errors. Finally, we visualize the simulated data.

### Simulation setup

The configuration is defined using an associative dictionary between the expected parameters and the values of its parameters. The description of the parameters is available on the [online help](https://swot-simulator.readthedocs.io/en/latest/generated/swot_simulator.settings.Parameters.html#swot_simulator.settings.Parameters).

This array can be loaded from a Python file using the [eval_config_file](https://swot-simulator.readthedocs.io/en/latest/generated/swot_simulator.settings.eval_config_file.html#swot_simulator.settings.eval_config_file) method. But it can also be declared directly in Python code.

In [ ]:
import os

In [ ]:
configuration = dict(
    # The swath contains in its centre a central pixel divided in two by the
    # reference ground track.
    central_pixel = True,
    # Distance, in km, between two points along track direction.
    delta_al = 2.0,
    # Distance, in km, between two points across track direction.
    delta_ac = 2.0,
    # Distance, in km, between the nadir and the center of the first pixel of the
    # swath
    half_gap = 2.0,
    # Distance, in km, between the nadir and the center of the last pixel of the
    # swath
    half_swath = 70.0,
    # Limits of SWOT swath requirements. Measurements outside the span will be set
    # with fill values.
    requirement_bounds = [10, 60],
    # Ephemeris file to read containing the satellite's orbit.
    ephemeris="orbaltika.txt",
    ephemeris_cols = [0, 1, 2],
    cycle_duration=35,
    height=790000,
    nadir = True,
    swath = False,
#    area = [-78, -68, 30, 40],#Gulf Stream
    # File containing spectrum of instrument error
    error_spectrum=os.path.join("..", "data", "error_spectrum.nc"),
    # KaRIN file containing spectrum for several SWH
    karin_noise=os.path.join("..", "data", "karin_noise_v2.nc"),
    # The plug-in handling the SSH interpolation under the satellite swath.
    #ssh_plugin = TODO
)

We create the parameter object for our simulation.

In [3]:
import swot_simulator.settings

parameters = swot_simulator.settings.Parameters(configuration)

---
**Note**

The [Parameter](https://swot-simulator.readthedocs.io/en/latest/generated/swot_simulator.settings.Parameters.html#swot_simulator.settings.Parameters) class exposes the [load_default](https://swot-simulator.readthedocs.io/en/latest/generated/swot_simulator.settings.Parameters.load_default.html#swot_simulator.settings.Parameters.load_default) method returning the default parameters of the simulation:

```python
parameters = swot_simulator.settings.Parameters.load_default()
```
---

### SSH interpolation

The written configuration allows us to simulate a swath. However, the interpolation of the SSH under the satellite swath remains undefined. If you don't need this parameter, you can skip this setting.

For our example, we use the SSH of the CMEMS grids provided on the Pangeo site.

In [4]:
dir="/work/ALT/swot/aval/wisa/tmp/eNATL60-BLB002-SSH-1h"

In [5]:
!ls $dir

nav_lat  sossheig	time_centered_bounds  time_counter_bounds
nav_lon  time_centered	time_counter


In [6]:
import os
import re
import numpy as np
# import pyinterp.backends.xarray
import pyinterp
import xarray as xr
import zarr
import time
import dask.array as da
import logging

from swot_simulator.plugins.ssh.mitgcm import MITGCM, _time_interp

LOGGER = logging.getLogger(__name__)

def _spatial_interp_2D(z_model: da.array, x_model: da.array, y_model: da.array,
                    x_sat: np.ndarray, y_sat: np.ndarray):
    mesh = pyinterp.RTree(dtype="float32")
    x, y, z = (), (), ()

    start_time = time.time()
    x_face = x_model.compute()
    y_face = y_model.compute()
    # We test if the face covers the satellite positions.
    ix0, ix1 = x_face.min().values, x_face.max().values
    iy0, iy1 = y_face.min().values, y_face.max().values

    box = pyinterp.geodetic.Box2D(pyinterp.geodetic.Point2D(ix0, iy0),
                                    pyinterp.geodetic.Point2D(ix1, iy1))
    mask = box.covered_by(x_sat, y_sat)
    if not np.any(mask == 1):
        print ("no covers")
    del box, mask

    # The undefined values are filtered
    z_face = z_model.compute()
    defined = ~np.isnan(z_face)

    # The tree is built and the interpolation is calculated
    x = x_face.values[defined]
    y = y_face.values[defined]

    coordinates = np.vstack((x, y)).T
    del x, y

    z = z_face.values[defined]
    LOGGER.info("loaded %d MB in %.2fs",
                 (coordinates.nbytes + z.nbytes) // 1024**2,
                 time.time() - start_time)
    start_time = time.time()
    mesh.packing(coordinates, z)
    LOGGER.info("mesh build in %.2fs", time.time() - start_time)

    del coordinates, z

    start_time = time.time()
    z, _ = mesh.inverse_distance_weighting(np.vstack(
        (x_sat, y_sat)).T.astype("float32"),
                                           within=True,
                                           k=11,
                                           radius=8000,
                                           num_threads=1)
    LOGGER.debug("interpolation done in %.2fs", time.time() - start_time)
    del mesh
    return z.astype("float32")  

class Enatl60(MITGCM):
    """
    Interpolation of the SSH eNATL60 products in zarr format.
    """

    def __init__(self, path): #xc: xr.DataArray, yc: xr.DataArray, eta: xr.DataArray):
        self.path = path
        with xr.open_zarr(self.path) as ds:
            self.lon = ds.nav_lon
            self.lat = ds.nav_lat
            self.ssh = ds.sossheig
            self.ts = ds.time_counter.data.astype("datetime64[us]")
        self.dt = self._calculate_dt(self.ts)

    def interpolate(self, lon: np.ndarray, lat: np.ndarray,
                    dates: np.ndarray) -> np.ndarray:
        """Interpolate the SSH for the given coordinates
        copy from the mit_gcm.interpolate function, just to overwrite spatial_interp function
        """
        first_date = self._grid_date(dates[0], -1)
        last_date = self._grid_date(dates[-1], 1)

        if first_date < self.ts[0] or last_date > self.ts[-1]:
            raise IndexError(
                f"period [{first_date}, {last_date}] is out of range: "
                f"[{self.ts[0]}, {self.ts[-1]}]")

        # Mask for selecting data covering the time period provided.
        mask = (self.ts >= first_date) & (self.ts <= last_date)

        LOGGER.debug("fetch data for %s, %s", first_date, last_date)

        # 4D cube representing the data necessary for interpolation.
        frame = self.ssh[mask]

        # Spatial interpolation of the SSH on the different selected grids.
        start_time = time.time()
        layers = []
        for index in range(len(frame)):
            layers.append(
                _spatial_interp_2D(frame[index, :], self.lon, self.lat, lon, lat))

        # Time interpolation of the SSH.
        layers = np.stack(layers)
        LOGGER.debug("interpolation completed in %.2fs for period %s, %s",
                     time.time() - start_time, first_date, last_date)
        return _time_interp(self.ts[mask].astype("int64"), layers,
                            dates.astype("datetime64[us]").astype("int64"))

In [7]:
parameters.ssh_plugin = Enatl60(dir)

### Initiating orbit propagator.

Initialization is simply done by [loading](https://swot-simulator.readthedocs.io/en/latest/generated/swot_simulator.orbit_propagator.load_ephemeris.html#swot_simulator.orbit_propagator.load_ephemeris) the ephemeris file. The satellite's one-day pass is taken into account in this case.

In [8]:
import swot_simulator.orbit_propagator


with open(parameters.ephemeris, "r") as stream:
    orbit = swot_simulator.orbit_propagator.calculate_orbit(parameters, stream)

In [9]:
import numpy
first_date = numpy.datetime64("2009-07-01")
iterator = orbit.iterate(first_date)
cycle_number, pass_number, date = next(iterator)
cycle_number, pass_number, date

(1, 1, numpy.datetime64('2009-07-01'))

### Generate the positions under the swath.

To perform this task, the following function is implemented.

> If the position of the pass is outside the area of interest (`parameters.area`),
> the generation of the pass can return `None`.

In [10]:
def generate_one_track(pass_number, date, orbit):
    # Compute the spatial/temporal position of the satellite
    track = swot_simulator.orbit_propagator.calculate_pass(pass_number, orbit, parameters)

    # If the pass is not located in the area of interest (parameter.area)
    # the result of the generation can be null.
    if track is None:
        return None

    # Set the simulated date
    track.time = date
    
    return track

### Interpolate SSH

Interpolation of the SSH for the space-time coordinates generated by the simulator.

In [11]:
def interpolate_ssh(parameters, track):
    swath_time = numpy.repeat(track.time, track.lon.shape[1]).reshape(track.lon.shape)
    ssh = parameters.ssh_plugin.interpolate(track.lon.flatten(),
                                            track.lat.flatten(),
                                            swath_time.flatten())
    return ssh.reshape(track.lon.shape)

 ### Create the swath dataset
 
 Generation of the simulated swath. The function returns an xarray dataset for the half-orbit generated.

In [12]:
import swot_simulator.product_specification


def generate_dataset(cycle_number,
                     pass_number,
                     track,
                     ssh,
                     complete_product=False):
    product = swot_simulator.product_specification.Nadir(track)
    # Mask to set the measurements outside the requirements of the mission to
    # NaN.
    mask = track.mask()
    ssh *= mask
    product.ssh(ssh)
    product.simulated_true_ssh(ssh)
    return product.to_xarray(cycle_number, pass_number, complete_product)

### Swath generation.

Now we can combine the different components to generate the swath.

In [13]:
import dask


def generate_swath(cycle_number, pass_number, date, parameters, orbit):
    client = dask.distributed.get_client()
    # Compute swath positions
    track = dask.delayed(generate_one_track)(pass_number, date, orbit)
    # Interpolate SSH
    ssh = dask.delayed(interpolate_ssh)(parameters, track)
    # Finally generate the dataset
    return dask.delayed(generate_dataset)(
        cycle_number, pass_number, track, ssh,
        parameters.complete_product).compute()

The simulator calculation can be distributed on a Dask cluster.

In [14]:
import dask.distributed

# A local cluster is used here.
cluster = dask.distributed.LocalCluster()
client = dask.distributed.Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:45055 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 64.42 GB


In [15]:
parameters_ = client.scatter(parameters)
orbit_ = client.scatter(orbit)

In [16]:
future = client.submit(generate_swath,
                       cycle_number,
                       pass_number,
                       date,
                       parameters_,
                       orbit_)
ds = client.gather(future)
ds

ValueError: different number of dimensions on data and dims: 2 vs 1

To calculate a trace set you can use the following code

    futures = []
    for cycle_number, pass_number, date in iterator:
        futures.append(client.submit(generate_swath,
                                     cycle_number,
                                     pass_number,
                                     date,
                                     error_generator_,
                                     orbit_,
                                     parameters))
    client.gather(futures)
    
### Visualization

In [ ]:
import matplotlib.pyplot
import cartopy.crs
import cartopy.feature
%matplotlib inline

Selection of a reduced geographical area for visualization.

Simulated SSH measurements (Interpolated SSH and simulated instrumental errors).

In [ ]:
fig = matplotlib.pyplot.figure(figsize=(24,12))
ax = fig.add_subplot(1, 1, 1, projection=cartopy.crs.PlateCarree())
contourf = ax.contourf(ds.longitude, ds.latitude, ds.ssh_karin,
            transform=cartopy.crs.PlateCarree(), levels=255, cmap='jet')
fig.colorbar(contourf, orientation='vertical')
ax.set_extent([60, 69, -50, -40], crs=cartopy.crs.PlateCarree())
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
#ax.add_feature(cartopy.feature.LAND)
#ax.add_feature(cartopy.feature.COASTLINE)
#ax.coastlines()

Simulated KaRIN instrumental noise.

In [ ]:
for item in selected.variables:
    if item.startswith("simulated_error"):
        variable = selected.variables[item]
        fig = matplotlib.pyplot.figure(figsize=(18, 8))
        ax = fig.add_subplot(1, 1, 1)
        image = ax.imshow(variable.T,
                          extent=[0, len(selected.num_lines), -70, 70],
                          cmap='jet')
        ax.set_title(variable.attrs['long_name'] + "(" +
                     variable.attrs['units'] + ")")
        ax.set_xlabel("num_lines")
        ax.set_ylabel("num_pixels")
        fig.colorbar(image,
                     orientation='vertical',
                     fraction=0.046 * 70 / 250,
                     pad=0.04)